In [1]:
!pip install transformers
!pip install youtokentome

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import math
import numpy as np
import pandas as pd

import random
import json
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import youtokentome as yttm
import torch.nn.functional as F


from tqdm import tqdm

from matplotlib import pyplot as plt
from transformers import AutoTokenizer

In [4]:
%cd '/content/drive/MyDrive/комплинг/дз3'

/content/drive/MyDrive/комплинг/дз3


In [5]:
class TrainDataset(Dataset):
   
    def __init__(self, path_to_file, sep='\t'):
        super().__init__()
        
        self.path_to_file = path_to_file
        self.sep = sep
        
        self.data = self.load_data()
        self.train, self.val, self.test = self.split_dataset()
        self.dataframe = pd.DataFrame({'question': self.data[0], 'answer': self.data[1]})
    

    def load_data(self):        
        data_seq = []
        question, answer = [], []
        counter = 0
        with open(self.path_to_file,) as f:
            for line in f:
                if counter == 400000:
                  break
                data_seq.append(line.strip().split(self.sep))
                counter += 1
                # data = line.strip().split(self.sep)
                # question.append(data[0])
                # answer.append(data[1])
        return data_seq
    

    def prepare_tokenization(self):

      with open('for_bpe.txt', 'w', encoding='utf-8') as f:
        for que, answ in self.train:
            f.write(que + '\t' + answ + '\n')
      

    def split_dataset(self):
      
      train, val, test = np.split(np.array(self.data), [int(.8 * len(self.data)), int(.9 * len(self.data))])

      return train, val, test

In [6]:
dataset = TrainDataset('train.txt')
train, dev, test = dataset.train, dataset.val, dataset.test

In [7]:
dataset.prepare_tokenization()

In [8]:
vocab_size = 30_000
model_path = 'pretrained_bpe_lm.model'
yttm.BPE.train(data='for_bpe.txt', vocab_size=vocab_size, model=model_path)

In [9]:
class Tokenizer:
    
    def __init__(self, model_path, max_length):
      
        self.tokenizer = yttm.BPE(model=model_path)
        
        self.tokenizer.pad_token = '<pad>'
        self.pad_index = 0
        self.max_length = max_length


    def tokenize(self, text):
        """
        В этом методе нужно разделить строку текста на токены
        """
        ...

        return self.tokenizer.encode(text, bos=True, eos=True)

        
    def padding(self, tokens_indices):
        """
        В этом методе нужно сделать длину tokens_indices равной self.max_length
        """
        ...        
        padded_seq = F.pad(torch.tensor(tokens_indices), (0,  self.max_length - len(tokens_indices)), value=self.pad_index)

        return padded_seq

    
    def __call__(self, text):
        """
        В этом методе нужно перевести строку с текстом в вектор с индексами слов нужно размера (self.max_length)
        """
        ...

        return torch.LongTensor(self.padding(self.tokenize(text)))

    
    def no_eos(self, text):
         return torch.LongTensor(self.padding(self.tokenizer.encode(text, bos=True, eos=False)))


    def collate(self, batch):
        
        question = torch.stack([self.__call__(el[0]) for el in batch])

        answer = torch.stack([self.__call__(el[1]) for el in batch])

        answer_no_eos = torch.stack([self.no_eos(el[1]) for el in batch])

        return question, answer, answer_no_eos

In [10]:
tokenizer = Tokenizer(model_path, 64)

In [11]:
train_loader = DataLoader(train, batch_size=64, shuffle=True, collate_fn=lambda x: tokenizer.collate(x))
dev_loader = DataLoader(dev, batch_size=64, shuffle=True, collate_fn=lambda x: tokenizer.collate(x))
test_loader = DataLoader(test, batch_size=64, shuffle=True, collate_fn=lambda x: tokenizer.collate(x))

In [12]:
for x, y, y_no_eos in train_loader:
    break

In [13]:
x.shape, y.shape, y_no_eos.shape

(torch.Size([64, 64]), torch.Size([64, 64]), torch.Size([64, 64]))

In [14]:
progress_bar = tqdm(total=len(dev_loader.dataset), desc='Evaluation')

for x, y, _ in dev_loader:
    progress_bar.update(x.size(0))
    
progress_bar.close()

Evaluation: 100%|██████████| 40000/40000 [00:03<00:00, 11566.55it/s]


In [15]:
dev_loader.dataset

array([['можа хватит уже чревоугодничать и будем прелюбодействовать,а?)',
        'погоди., ещё не допили. дринк!. а вообще назрел вопрос.!'],
       ['самы дарогой вит спорта', 'для нашей страны футбол.'],
       ['отправьте мне фото которое является самым красивым в мире',
        'для каждого свое. мне нравится например это ---&gt;'],
       ...,
       ['сударыня вы пишете душою?',
        'сердцем . а я вот спросить хотела , а кому это надо?'],
       ['эй? у тебя есть душа?', 'после праздников найти надо будет.'],
       ['как перестать слушать "медузу" ?', 'нажать кнопочку "выкл."']],
      dtype='<U64')

## Encoder CNN

In [16]:
INPUT_DIM = vocab_size
OUTPUT_DIM = vocab_size
KERNEL_SIZE = 2
ENC_EMB_DIM = 100
DEC_EMB_DIM = 100
HID_DIM = 512
N_LAYERS = 1

In [18]:
class ConvBlock(nn.Module):
   def __init__(self, hid_dim, device):
      super().__init__()

      # сверточные слои
      self.conv1 = nn.Conv1d(in_channels=hid_dim, 
                               out_channels=hid_dim,
                               kernel_size=3,
                               padding = 1)
        
      self.conv2 = nn.Conv1d(in_channels=hid_dim, 
                               out_channels=hid_dim,
                               kernel_size=5,
                               padding = 2)

      self.batch_norm = nn.BatchNorm1d(hid_dim)
      self.activ = nn.ReLU()
      self.scale = torch.sqrt(torch.FloatTensor([0.4])).to(device)
      self.device = device
    
   def forward(self, to_conv_perm):
      
      # сверточные слои
      conv1 = self.conv1(to_conv_perm)  # [batch, hid_dim, len_sent]
      conv1 = self.batch_norm(conv1)
      # при gelu, out_chanels должен задаваться как 2 * hid_dim, так как в ней зашито деление на 2
      conv1 = self.activ(conv1)  # [batch, hid_dim, len_sent]
      conv_after_resid = (to_conv_perm + conv1) * self.scale  # [batch, hid_dim, len_sent]


      conv2 = self.conv2(conv_after_resid)  # [batch, hid_dim, len_sent]
      conv2 = self.batch_norm(conv2)
      conv2 = self.activ(conv2)  # [batch, hid_dim, len_sent]
      conv_after_2resid = (conv_after_resid + conv2) * self.scale  # [batch, hid_dim, len_sent]

        
      # смена размерности для линейного слоя
      to_out_lin = conv_after_2resid.permute(0, 2, 1)  # [batch, len_sent, hid_dim]

      return to_out_lin

In [19]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, kernel_s, device, max_length=64):
        super().__init__()

        # инициализируем эмбеддинги для токенов и их позиций (от первого токена до последнего)
        self.position_embedding = nn.Embedding(max_length, emb_dim)
        self.token_embedding = nn.Embedding(input_dim, emb_dim)
        
        # линейные слои
        self.linear_toblock = nn.Linear(emb_dim, hid_dim)
        self.linear_outblock = nn.Linear(hid_dim, emb_dim)

        # сверточные слои
        self.scale = torch.sqrt(torch.FloatTensor([0.4])).to(device)
        self.conv_b = ConvBlock(hid_dim, device)
        self.device = device
      
    def forward(self, b_elem):

        batch_size = b_elem.shape[0]  # размер батча 
        src_len = b_elem.shape[1]    # длина последовательности

        # генерация матрицы тензора для позиций слов: [batch, len_sent]
        posintion_t = torch.stack([torch.arange(0, src_len) for pos in range(batch_size)]).to(self.device)
        # получаем эмбеддинги: [batch, len_sent, embed_dim]
        tok_emb = self.token_embedding(b_elem.to(self.device))


        pos_emb = self.position_embedding(posintion_t)

        # elemenwise sum: [batch, len_sent, embed_dim]
        elem_wise = tok_emb + pos_emb  # тут еще можно добавить дропаут
        
        # линейные слой: [batch, len_sent, hid_dim]
        to_block = self.linear_toblock(elem_wise)

        # входная размерность в conv - (N,C,L), где L - длина последоваетльности, С - число каналов 
        # [batch, hid_dim, len_sent]
        to_conv_perm = to_block.permute(0, 2, 1)

        

        # # сверточные слои       
        to_out_lin = self.conv_b(to_conv_perm)

        # линейный слой
        out_lin = self.linear_outblock(to_out_lin)  # [batch, len_sent, emb_dim]
        
        # residual connection
        res_con_out = (out_lin + elem_wise) * self.scale  # [batch, len_sent, emb_dim]

        return out_lin, res_con_out

In [ ]:
# encoder = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, KERNEL_SIZE)
# out_lin, res_con = encoder(x)

## Decoder CNN

In [20]:
class Attention(nn.Module):
    def __init__(self,emb_size,hid_size, device):

        super(Attention,self).__init__()
        self.emb2hid=nn.Linear(emb_size,hid_size)
        self.hid2emb=nn.Linear(hid_size,emb_size)
        self.scale=torch.sqrt(torch.FloatTensor([0.4])).to(device)
    
    def forward(self,dec_conved,embedd,en_conved,en_combined):

        dec_conved = dec_conved.permute(0,2,1)
        dec_conved_emb = self.hid2emb(dec_conved)

        Q = (dec_conved_emb + embedd) * self.scale
        energy = torch.matmul(Q, en_conved.permute(0,2,1))
        a = F.softmax(energy, dim=2)

        context = torch.matmul(a, en_combined)
        context = self.emb2hid(context)
        conved = (context + dec_conved) * self.scale

        return a, conved.permute(0,2,1)

In [21]:
class ConvBlock_Decod(nn.Module):
   def __init__(self, emb_dim, hid_dim, pad_param1, pad_param2, device):
      super().__init__()

      self.pad_param1 = pad_param1
      self.pad_param2 = pad_param2

      # сверточные слои
      self.conv1 = nn.Conv1d(in_channels=hid_dim, 
                               out_channels=hid_dim,
                               kernel_size=3+2,
                               padding = self.pad_param1)
        
      self.conv2 = nn.Conv1d(in_channels=hid_dim, 
                               out_channels=hid_dim,
                               kernel_size=5,
                               padding = self.pad_param2)
      
      self.batch_norm = nn.BatchNorm1d(hid_dim)

      self.attent = Attention(emb_dim, hid_dim, device)

      self.scale = torch.sqrt(torch.FloatTensor([0.4])).to(device)

      self.activ = nn.ReLU()
      self.device = device
    

   def forward(self, padded_seq, to_conv_perm, elem_wise, enc_output, enc_res_output):
        # сверточные слои
        conv1 = self.conv1(padded_seq)  # [batch, hid_dim, len_sent]
        conv1 = self.batch_norm(conv1)
        conv1 = self.activ(conv1)  # [batch, hid_dim, len_sent]

        attentioned, out_conv = self.attent(conv1,elem_wise,enc_output,enc_res_output)

        conv_after_resid = (to_conv_perm + out_conv) * self.scale  # [batch, hid_dim, len_sent]

        conv2 = self.conv2(conv_after_resid)  # [batch, hid_dim, len_sent]
        conv2 = self.batch_norm(conv2)
        conv2 = self.activ(conv2)  # [batch, hid_dim, len_sent]

        attentioned2, out_conv2 = self.attent(conv2,elem_wise,enc_output,enc_res_output)
        conv_after_2resid = (conv_after_resid + out_conv2) * self.scale  # [batch, hid_dim, len_sent]
        return conv_after_2resid, attentioned2

In [22]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, kernel_s, device, pad_idx=0, max_length=64):
        super().__init__()

        self.pad_idx = pad_idx
        self.pad_param1 = 1
        self.pad_param2 = 2
        self.emb_dim = emb_dim

        self.kern_s = kernel_s

        # инициализируем эмбеддинги для токенов и их позиций (от первого токена до последнего)
        self.position_embedding = nn.Embedding(max_length, emb_dim)
        self.token_embedding = nn.Embedding(output_dim, emb_dim)
        
        # линейные слои
        self.linear_toblock = nn.Linear(emb_dim, hid_dim)
        self.linear_outblock = nn.Linear(hid_dim, emb_dim)
        self.liner_out = nn.Linear(emb_dim, output_dim)

        self.conv_blocks = ConvBlock_Decod(self.emb_dim, hid_dim, self.pad_param1, self.pad_param2, device)

        self.dropout = nn.Dropout(0.05)
        self.device = device
  

    def forward(self, b_elem, enc_output, enc_res_output):

        batch_size = b_elem.shape[0]  # размер батча 
        src_len = b_elem.shape[1]    # длина последовательности

        # генерация матрицы тензора для позиций слов: [batch, len_sent]
        posintion_t = torch.stack([torch.arange(0, src_len) for pos in range(batch_size)]).to(self.device)

        # получаем эмбеддинги: [batch, len_sent, embed_dim]
        tok_emb = self.token_embedding(b_elem.to(self.device))
        pos_emb = self.position_embedding(posintion_t)

        # elemenwise sum: [batch, len_sent, embed_dim]
        elem_wise = tok_emb + pos_emb  # тут еще можно добавить дропаут
        
        # линейные слой: [batch, len_sent, hid_dim]
        to_block = self.linear_toblock(elem_wise)

        # входная размерность в conv - (N,C,L), где L - длина последоваетльности, С - число каналов 
        # [batch, hid_dim, len_sent]
        to_conv_perm = to_block.permute(0, 2, 1)

        # делаем паддинги
        padding = torch.zeros(to_block.shape[0], to_block.shape[2], self.pad_param1+1).fill_(self.pad_idx).to(self.device)
        # срезаем маркер конца предложения
        padded_seq = torch.cat((padding, to_conv_perm), dim=2)

        # # сверточные слои
        conv_after_2resid, attentioned2 = self.conv_blocks(padded_seq, to_conv_perm, elem_wise, enc_output, enc_res_output)
        
        # смена размерности для линейного слоя
        to_out_lin = conv_after_2resid.permute(0, 2, 1)  # [batch, len_sent, hid_dim]
        
        # линейный слой
        out_lin = self.linear_outblock(to_out_lin)  # [batch, len_sent, emb_dim]
        
        drop = self.dropout(out_lin)
        fn = self.liner_out(drop)
        # # residual connection
        # res_con_out = out_lin + elem_wise  # [batch, len_sent, emb_dim]

        return fn, attentioned2

In [ ]:
# dencoder = Decoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, KERNEL_SIZE)
# out, att = dencoder(y, out_lin, res_con)

In [ ]:
# out.shape

In [ ]:
# att.shape

In [23]:
class CNN_Seq2Seq(nn.Module):

  def __init__(self, input_dim, output_dim, enc_emb_dim, dec_emb_dim, hid_dim, kernel_s, device, pad_idx=0, max_length=64):
        super().__init__()
        self.encoder = Encoder(input_dim, enc_emb_dim, hid_dim, kernel_s, device)
        self.decoder = Decoder(output_dim, dec_emb_dim, hid_dim, kernel_s, device)
  
  def forward(self, sourse, target):

        sourse_output, sourse_output_resid = self.encoder(x)

        target_output, target_atten = self.decoder(target, sourse_output, sourse_output_resid)

        return target_output, target_atten

In [24]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [25]:
INPUT_DIM = vocab_size
OUTPUT_DIM = vocab_size
KERNEL_SIZE = 2
ENC_EMB_DIM = 300
DEC_EMB_DIM = 300
HID_DIM = 512
N_LAYERS = 1

In [40]:
model = CNN_Seq2Seq(INPUT_DIM, OUTPUT_DIM, ENC_EMB_DIM, DEC_EMB_DIM, HID_DIM, KERNEL_SIZE, device)

In [41]:
model.to(device)

CNN_Seq2Seq(
  (encoder): Encoder(
    (position_embedding): Embedding(64, 300)
    (token_embedding): Embedding(30000, 300)
    (linear_toblock): Linear(in_features=300, out_features=512, bias=True)
    (linear_outblock): Linear(in_features=512, out_features=300, bias=True)
    (conv_b): ConvBlock(
      (conv1): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(512, 512, kernel_size=(5,), stride=(1,), padding=(2,))
      (batch_norm): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activ): ReLU()
    )
  )
  (decoder): Decoder(
    (position_embedding): Embedding(64, 300)
    (token_embedding): Embedding(30000, 300)
    (linear_toblock): Linear(in_features=300, out_features=512, bias=True)
    (linear_outblock): Linear(in_features=512, out_features=300, bias=True)
    (liner_out): Linear(in_features=300, out_features=30000, bias=True)
    (conv_blocks): ConvBlock_Decod(
      (conv1): Conv1d(512, 512, kernel_si

In [44]:
criterion = torch.nn.CrossEntropyLoss(ignore_index=0).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-4)

In [45]:
epochs = 1

losses = []
predictions = []

epoch_score_train = []
epoch_score_test = []

In [46]:
def evaluating(dataloader, model, criterion, optimizer, n_epoch):
    
    losses = []
    
    print('Epoch #{}\n'.format(n_epoch+1))
    
    model.eval()

    try:

      progress_bar = tqdm(total=len(dataloader.dataset) / 64, desc='Epoch {}'.format(n_epoch + 1))

      for x, y, y_no_eos in dataloader:

            x = x.to(device)
            y = y.to(device)
            y_no_eos = y_no_eos.to(device)
            
            
            output, _ = model(x, y_no_eos[:,:-1])
            output_dim = output.shape[-1]
            output = output.contiguous().view(-1, output_dim)
            y = y[:,1:].contiguous().view(-1)
            
            loss = criterion(output, y)
            
            # torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
            
            # optimizer.step()
            
            losses.append(loss.item())
          
            progress_bar.update()
            progress_bar.set_postfix(loss=np.mean(losses[-100:]))

      progress_bar.close()

    except KeyboardInterrupt:

        progress_bar.close()
        # break
    
    print(f'\tValidation Loss: {np.mean(losses[-100:])}')
          
    return np.mean(losses[-100:])

In [47]:
def training(dataloader, model, criterion, optimizer, n_epoch, clip):
    
    losses = []
    
    print('Epoch #{}\n'.format(n_epoch+1))
    
    model.train()

    try:

      progress_bar = tqdm(total=len(dataloader.dataset) / 64, desc='Epoch {}'.format(n_epoch + 1))

      for x, y, y_no_eos in dataloader:

            x = x.to(device)
            y = y.to(device)
            y_no_eos = y_no_eos.to(device)
            
            optimizer.zero_grad()
            
            output, _ = model(x, y_no_eos[:,:-1])
            
            output_dim = output.shape[-1]
            
            output = output.contiguous().view(-1, output_dim)
            y = y[:,1:].contiguous().view(-1)
            
            loss = criterion(output, y)
            loss.backward()
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
            
            optimizer.step()
            
            losses.append(loss.item())
          
            progress_bar.update()
            progress_bar.set_postfix(loss=np.mean(losses[-100:]))

      progress_bar.close()

    except KeyboardInterrupt:

        progress_bar.close()
        # break
    
    print(f'\tTrain Loss: {np.mean(losses[-100:])}')
          
    return np.mean(losses[-100:])

In [48]:
def generate(sentence, model, bos_index=2, eos_index=3, max_sequence=45):

    tokenized = tokenizer.tokenize([sentence])

    # добавляем тег начала предложения
    tokenized[0].insert(0, bos_index)

    # print(tokenized)
    x = torch.tensor(tokenized).long().to(device)

    model.eval()

    with torch.no_grad():
        encoder_conved, encoder_combined = model.encoder(x)


    trg_indexes = [[bos_index]]

    for i in range(max_sequence):
        trg_tensor = torch.LongTensor(trg_indexes).to(device)
        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, encoder_conved, encoder_combined)
        pred_token = output.argmax(2)[:, -1].item()
        trg_indexes[0].append(pred_token)
        if pred_token == eos_index:
            break

    trg_tokens = tokenizer.tokenizer.decode(trg_indexes[0])

    return ' '.join(trg_tokens)

In [49]:
generate('Что мне купить сегодня вечером на ужин?', new_m.to(device), bos_index=2, eos_index=3, max_sequence=20)

'<BOS> у в не<EOS>'

In [50]:
n_epoch = 7
clip = 1

try_sent = 'Что мне купить сегодня вечером на ужин?'

for epoch in range(n_epoch):
    
    # print('Epoch #{}\n'.format(n_epoch+1))
    try:
    
      
      train_loss = training(train_loader, model, criterion, optimizer, epoch, clip)
      valid_loss = evaluating(dev_loader, model, criterion, optimizer, epoch)
      torch.save(model.state_dict(), 'cnntry2-model.pt')
      print(generate(try_sent, model, bos_index=2, eos_index=3, max_sequence=20))

    except KeyboardInterrupt:
      break

Epoch #1



Epoch 1: 100%|██████████| 5000/5000.0 [12:42<00:00,  6.56it/s, loss=0.0295]


	Train Loss: 0.029535379901062697
Epoch #1



Epoch 1: 100%|██████████| 625/625.0 [00:31<00:00, 20.11it/s, loss=0.0291]


	Validation Loss: 0.02907277975231409
<BOS>самыйдоста подумали подумали подумали подумализрос шарапо мети шарапо подводит исполнитель разговаривает сортизрос переубедчч полюбому обучлс
Epoch #2



...

	Validation Loss: 0.009173334959195928
<BOS> небо? свадьбы?ever месяца?руете? свадьбы? волосами? ума? вашему, безал знаете?) романтики? ег расшифровы лансер телосло вашему,㋛ романтики? месяца?
Epoch #7



Epoch 7: 100%|██████████| 5000/5000.0 [12:44<00:00,  6.54it/s, loss=0.000362]


	Train Loss: 0.0003619438811665532
Epoch #7



Epoch 7: 100%|██████████| 625/625.0 [00:31<00:00, 20.00it/s, loss=0.0234]


	Validation Loss: 0.023414298074203543
<BOS>ги? зарплату?вает?) готовлю отдал носить? выходилигарита зарплата? носить? питомцы.как лечи таки,.если菲ки"?ваете,вает?) школе,


In [56]:
torch.save(model.state_dict(), 'model_seq2seq.pt')

In [57]:
new_m = CNN_Seq2Seq(INPUT_DIM, OUTPUT_DIM, ENC_EMB_DIM, DEC_EMB_DIM, HID_DIM, KERNEL_SIZE, device)
new_m.load_state_dict(torch.load('/content/drive/MyDrive/комплинг/дз3/model_seq2seq.pt'))
new_m.eval()

CNN_Seq2Seq(
  (encoder): Encoder(
    (position_embedding): Embedding(64, 300)
    (token_embedding): Embedding(30000, 300)
    (linear_toblock): Linear(in_features=300, out_features=512, bias=True)
    (linear_outblock): Linear(in_features=512, out_features=300, bias=True)
    (conv_b): ConvBlock(
      (conv1): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(512, 512, kernel_size=(5,), stride=(1,), padding=(2,))
      (batch_norm): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activ): ReLU()
    )
  )
  (decoder): Decoder(
    (position_embedding): Embedding(64, 300)
    (token_embedding): Embedding(30000, 300)
    (linear_toblock): Linear(in_features=300, out_features=512, bias=True)
    (linear_outblock): Linear(in_features=512, out_features=300, bias=True)
    (liner_out): Linear(in_features=300, out_features=30000, bias=True)
    (conv_blocks): ConvBlock_Decod(
      (conv1): Conv1d(512, 512, kernel_si

In [54]:
sentences = [
    'если',
    'кто',
    'что делать, если',
    'почему мне так плохо',
    'как купить машину',
    'если я буду платить в кино, то что будет?',
    'как зовут моего кота?',
    'как выглядит совместная жизнь?',
    'почему меня презирают в школе?', 
    'где лучше всего отметить день рождения?',
    'как закончить магистратуру?', 
    'если завтра станет тепло, '
    'как мне усопокиться?',
    'что сделать, чтобы отметить новый год'
]

In [55]:
for sent in sentences:
  print('Вопрос: ', sent)
  print('Ответ: ', generate(sent, model, bos_index=2, eos_index=3, max_sequence=20).replace('<BOS>', ''))

Вопрос:  если
Ответ:  вым? обстоят (фото питомцы.как катего напле питомцы отдал галан живопи питомцыги?ником?) неизлечи пустое питомцы волосами?вает?)ите!
Вопрос:  кто
Ответ:  вым? обстоят (фото питомцы выходили предпочитаете? обстоятги? питомцыs, галан катего свит клички обстоят выходилитон, зарплата?вает?) обстоят
Вопрос:  что делать, если
Ответ:  вым? обстоят (фото питомцы.как катего напле питомцы отдал галан живопи питомцыги?ником?) неизлечи пустое питомцы волосами?вает?)ите!
Вопрос:  почему мне так плохо
Ответ:   носить?== зарплата? питомцыпродолжитеником?) отдал метивает?) готовлю живопи лечи питомцы.еслиником?) носить? выходили (фото питомцыпродолжите
Вопрос:  как купить машину
Ответ:  вым? обстоят (фото питомцы.как катего напле питомцы отдал галан живопи питомцыги?ником?) неизлечи пустое питомцы волосами?вает?)ите!
Вопрос:  если я буду платить в кино, то что будет?
Ответ:  вым? зарплата? (фото питомцы.как (фото (фото (фото фортепи опозна выбирает питомцы此 иметь? кличкипродолжит